In [ ]:
import numpy as np
import pandas as pd
import math
import pickle
import json
from numpy.random import rand, randint, exponential, normal, uniform, lognormal, dirichlet

In [ ]:
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
project_dir = '/content/drive/MyDrive/MASc_First_Year/MIE1612/Project'
data_dir = project_dir+ '/data/'

In [ ]:
# functions for dumping and loading generated data files
def pickle_dump(file_to_dump, file_dir):
  with open(file_dir, "wb") as fp:   #Pickling
    pickle.dump(file_to_dump, fp)

def pickle_load(file_dir):
  with open(file_dir, "rb") as fp:   # Unpickling
    return pickle.load(fp)

def json_dump(file_to_dump, file_dir):
    with open(file_dir, 'w') as fp:
        json.dump(file_to_dump, fp)

def json_load(file_dir):
    with open(file_dir) as f:
        return json.load(f)

### <b>1.Sets initialization</b>

B: set of bike-stations, B = {1, ..., B} <br>
S: set of scenarios, S = {1,...,S} or finit set of possible realization of the uncertainty.

In [ ]:
# Sets
Bset = 'A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V'.split(',')  # 22 Bike Stations
Bij_set = [(bike2, bike) for bike2 in Bset for bike in Bset]     # Bike pairs 

In [ ]:
# dump sets
pickle_dump(Bset, data_dir + 'Bset.txt')
pickle_dump(Bij_set, data_dir + 'Bij_set.txt')

#### <b>2.Deterministic parameters</b>

c: procurement cost per bike at each bike-station at the beginning of the service <br>
$v_i$: stock-out cost per bike at bike-station i $\in$ B <br>
$w_i$: time-waste cost per bike due to overflow at bike-station i $\in$ B <br>
$t_{ij}$: unit transshipment cost per bike transshiped from bike-station i to bike-station j, i,j $
\in$ B <br>
$k_i$: capacity of bike-station i $\in$ B <br>
$\xi_{ijk}$: rental demand from bike-station i to bike-station j in scenario s, i, j $\in$ B, s $\in$ S  <br>
$p_s \in [0,1]$: probability of scenario s $\in$ S, $\sum_{s=1}^{S} p_s$ = 1<br>



In [ ]:
c = 2         # unit procurement cost
v_i = 4       # stock-out cost
w_i = 8       # time-waste cost 
t_ij = 1      # unit transshipment cost 
deterministic_params = {'c': c, 'v_i': v_i, 'w_i': w_i, 't_ij': t_ij}

# capacity for each station 
k_i = [22, 10, 10, 17, 19, 12, 20, 8, 8, 20, 10, 19, 8, 10, 10, 10, 18, 10, 8, 12, 12, 10]
capacity_i = {Bset[index] : k_i[index] for index in range(len(Bset))}
capacity_i_constant = {station: 50 for station in Bset}   # for the constant capacity case (not using for now)

In [ ]:
# dump data
json_dump(deterministic_params, data_dir + 'deterministic_params.json')
json_dump(capacity_i, data_dir + 'capacity_i.json')
json_dump(capacity_i_constant, data_dir + 'capacity_i_constant.json')

#### <b>3. Stochastic generation</b>

In [ ]:
# demand generation function 
def generate_demand(distributions, Sset, Bij_set, mean_ij, sigma_ij):
  demScens = {}

  # Generate demand using log-normal distribution 
  # stochastic demand stored in form {((B_i, B_j), S_s) : demand}
  for distribution_choice, dist in distributions.items():
    print('--------------------%s--------------------'%distribution_choice)
    for sta_idx in tqdm(range(len(Bij_set))):
      station_pair = Bij_set[sta_idx]
      for scenario in Sset:
        # diagonal demands set to 0 
        if station_pair[0] == station_pair[1]:
          demand = 0
        elif distribution_choice == 'log-normal':
          demand = apply_upper_bound(int(dist(mean_ij[station_pair], sigma_ij[station_pair])))
        elif distribution_choice == 'normal':
          demand = apply_upper_bound(abs(int(dist(mean_ij[station_pair], sigma_ij[station_pair]))))
        elif distribution_choice == 'exponential':
          demand = apply_upper_bound(int(dist(mean_ij[station_pair])))
        elif distribution_choice == 'uniform':
          demand = apply_upper_bound(mean_ij[station_pair])
        else:
          assert(False)
          print('Invalid distribution!!!')

        demScens[(station_pair[0], station_pair[1], scenario)] = demand
    pickle_dump(demScens, data_dir + 'demScens_' + distribution_choice + '_' + str(len(Sset)) + '.dict')

def apply_upper_bound(number):
  return min(number, 5)

Run only once to generate consistent probabilities and distribution parameters 

In [ ]:
## Parameters
# mean_ij = {station_pair: randint(math.log(4), math.log(10)) for station_pair in Bij_set}
mean_ij = {station_pair: randint(1, 2) for station_pair in Bij_set}
sigma_ij = {station_pair: rand()/5 for station_pair in Bij_set}

## Distributions
distributions = {'log-normal': lognormal, 'normal': normal, 'uniform': uniform, 'exponential': exponential}

# Stochastic demand
np.random.seed(1612)

In [ ]:
scenario_num_set = [100,300,500,700,900,1000]  # scenario number set
for scenario_num in scenario_num_set:
  print('Generating for %d number of scenarios' % scenario_num)                                       
  Sset = ['S'+str(s) for s in range(scenario_num)]                 # Scenario sets
  prob = np.around(np.random.dirichlet(np.ones(len(Sset))), decimals = 7)      # not consistent?
  p_s = {scenario: prob[int(''.join(list(scenario)[1:]))]  for scenario in Sset}

  generate_demand(distributions, Sset, Bij_set, mean_ij, sigma_ij)

  # dump data iteratively, each for one scenario number 
  pickle_dump(Sset, data_dir+'Sset_' + str(scenario_num) + '.txt')
  json_dump(p_s, data_dir + 'p_s_' + str(len(p_s)) + '.json')